# Percent change

The assignment:
>1. Download this dataset of population estimates from the US Census Bureau.
1. Calculate the percent change in population for every county in the US from 2010 to 2014.
1. Round that change off to a single decimal point.
1. Sort it fastest growing to fastest shrinking. Print it to the screen but limit it to 50.

Just like always, we import `agate`.

In [1]:
import agate

Next, we import our population data into a table named `population`.

In [2]:
population = agate.Table.from_csv('population.csv')

Then, we print it to see its headers.

In [3]:
print(population)

|------------------+------------|
|  column          | data_type  |
|------------------+------------|
|  STNAME          | Text       |
|  CTYNAME         | Text       |
|  POPESTIMATE2010 | Number     |
|  POPESTIMATE2011 | Number     |
|  POPESTIMATE2012 | Number     |
|  POPESTIMATE2013 | Number     |
|  POPESTIMATE2014 | Number     |
|------------------+------------|



Here, we use the `PercentChange` function to calculate a percent change between `POPESTIMATE2010` and `POPESTIMATE2014`. That new data is then combined with the original data in a new table named `population_rate`.

In [4]:
population_rate = population.compute([
        ('change_1014', agate.PercentChange('POPESTIMATE2010', 'POPESTIMATE2014'))
    ])

Now, we want to print the first few rows to make sure it worked correctly.

In [5]:
population_rate.print_table(max_rows=10)

|----------+-----------------+-----------------+-----------------+-----------------+-----------------+------|
|  STNAME  | CTYNAME         | POPESTIMATE2010 | POPESTIMATE2011 | POPESTIMATE2012 | POPESTIMATE2013 | ...  |
|----------+-----------------+-----------------+-----------------+-----------------+-----------------+------|
|  Alabama | Autauga County  |          54,684 |          55,275 |          55,192 |          55,136 | ...  |
|  Alabama | Baldwin County  |         183,216 |         186,694 |         190,561 |         195,443 | ...  |
|  Alabama | Barbour County  |          27,336 |          27,225 |          27,169 |          26,978 | ...  |
|  Alabama | Bibb County     |          22,879 |          22,740 |          22,634 |          22,504 | ...  |
|  Alabama | Blount County   |          57,344 |          57,694 |          57,748 |          57,720 | ...  |
|  Alabama | Bullock County  |          10,886 |          10,623 |          10,589 |          10,605 | ...  |
|  Alabama

It worked, but it's a little messy. Let's only print the columns we want. To do that, we'll create a new table called `for_printing` by selecting columns from `population_rate`.

In [6]:
for_printing = population_rate.select(['STNAME', 'CTYNAME', 'change_1014'])

Why don't we also sort our fields while we're at it? We'll use the `order_by` function and create a new table called `sorted_counties`.

In [7]:
sorted_counties = for_printing.order_by('change_1014', reverse=True)

Now that we've done all that, let's print the first 50 entries.

In [8]:
sorted_counties.print_table(max_rows=50)

|---------------+---------------------+-----------------------|
|  STNAME       | CTYNAME             |          change_1014  |
|---------------+---------------------+-----------------------|
|  North Dakota | McKenzie County     | 71.94683346364347...  |
|  North Dakota | Williams County     | 42.33819164488548...  |
|  North Dakota | Mountrail County    | 26.62783171521035...  |
|  North Dakota | Stark County        | 24.71564078347636...  |
|  North Dakota | Dunn County         | 24.23044337757695...  |
|  Florida      | Sumter County       | 21.27864923054079...  |
|  Louisiana    | St. Bernard Parish  | 20.68974888574845...  |
|  Montana      | Richland County     | 18.87451222016841...  |
|  Texas        | Andrews County      | 17.80923491742500...  |
|  Texas        | Sterling County     | 17.76605101143359...  |
|  North Dakota | Divide County       | 17.26133076181292...  |
|  Virginia     | Fredericksburg city | 17.18750000000000...  |
|  Utah         | Wasatch County      | 

That worked, but those decimals are ugly. Let's round those off. To be honest, I'm not entirely sure how this works, but Matt says it does, so I'm going with it.

In [9]:
from decimal import Decimal

def round_popchange(row):
    return row['change_1014'].quantize(Decimal('0.1'))

rounded_change = sorted_counties.compute([
        ('pop_rounded', agate.Formula(agate.Number(), round_popchange))
    ])



Now we'll create a new table called `for_rate_printing` with the new rounded numbers. 

In [10]:
for_rate_printing = rounded_change.select(['STNAME', 'CTYNAME', 'pop_rounded'])

Then, we'll print the first 50 entries.

In [11]:
for_rate_printing.print_table(max_rows = 50)

|---------------+---------------------+--------------|
|  STNAME       | CTYNAME             | pop_rounded  |
|---------------+---------------------+--------------|
|  North Dakota | McKenzie County     |        71.9  |
|  North Dakota | Williams County     |        42.3  |
|  North Dakota | Mountrail County    |        26.6  |
|  North Dakota | Stark County        |        24.7  |
|  North Dakota | Dunn County         |        24.2  |
|  Florida      | Sumter County       |        21.3  |
|  Louisiana    | St. Bernard Parish  |        20.7  |
|  Montana      | Richland County     |        18.9  |
|  Texas        | Andrews County      |        17.8  |
|  Texas        | Sterling County     |        17.8  |
|  North Dakota | Divide County       |        17.3  |
|  Virginia     | Fredericksburg city |        17.2  |
|  Utah         | Wasatch County      |        17.1  |
|  Texas        | Hays County         |        16.9  |
|  North Dakota | Billings County     |        16.9  |
|  Georgia